## Data 구조 처리
학습용으로 사용하기 위한 이미지 데이터와 좌푯값 전처리 코드.

CSV 구성 형식
- 이미지 경로 , 관절좌표1, 관절좌표2, ... , 관절좌표14, MET Score

처리 형식
- 전체 데이터세트 이미지 최대 크기와 평균 크기 측정 -> 이미지 크기 조정
  크기 조정 비율에 따른 좌표값 재처리


In [1]:
import numpy as np # Numpy 배열 사용용도
import matplotlib.pyplot as plt # 이미지출력 용도
import pandas as pd # csv 파일을 pandas 프레임으로 호출 용도
import skimage
from skimage import io, transform # 이미지 불러오기 및 저장용도

# 주피터 노트북 상에 이미지를 출력하도록 설정
%matplotlib inline

In [2]:
# dataset CSV 파일을 불러와 pandas 프레임으로 저장한 뒤에 Numpy형식의 행렬로 변환하여 testset 변수에 저장
testset=pd.read_csv("/var/data/MET/dataset.csv",header=None).as_matrix()

testset

array([['/var/data/MET/images/walking/20.bmp', '[17, 146]', '[23, 115]',
        ..., '[40, 39]', '[35, 17]', 0.7],
       ['/var/data/MET/images/walking/19.bmp', '[22, 129]', '[12, 116]',
        ..., '[21, 38]', '[14, 16]', 0.7],
       ['/var/data/MET/images/walking/18.bmp', '[23, 153]', '[21, 129]',
        ..., '[23, 41]', '[20, 21]', 0.7],
       ..., 
       [ '/var/data/MET/images/miscellaneous occupational activity/machine work.sawing/18.jpg',
        '[-1, -1]', '[-1, -1]', ..., '[175, 61]', '[192, 1]', 1.8],
       [ '/var/data/MET/images/miscellaneous occupational activity/machine work.sawing/19.jpg',
        '[-1, -1]', '[98, 170]', ..., '[172, 59]', '[201, 8]', 1.8],
       [ '/var/data/MET/images/miscellaneous occupational activity/machine work.sawing/20.jpg',
        '[-1, -1]', '[-1, -1]', ..., '[194, 55]', '[185, 8]', 1.8]], dtype=object)

In [3]:
# 변수 초기화.
# 모든 x축 크기 총합, 모든 y축 크기 총합, 최대 x축 크기, 최대 y축 크기
allsum_x=allsum_y=max_x=max_y=0

for i in range(len(testset)): # testset 길이만큼 반복 // len(testset) = 100개 데이터
    img=skimage.io.imread(testset[i][0]) # 이미지 불러오기 (이미지 저장위치 불러오기)
    # img.shape의 구조는 "행렬의 rows, columns, channels의 갯수"
    
    if max_x<img.shape[0]: max_x=img.shape[0] # 최대 x축 길이값 저장
    if max_y<img.shape[1]: max_y=img.shape[1] # 최대 y축 길이값 저장
    allsum_x+=img.shape[0] # 모든 이미지의 x축 크기 총합
    allsum_y+=img.shape[1] # 모든 이미지의 y축 크기 총합
avg_x=allsum_x/len(testset) # 전체 이미지의 x축 크기 평균
avg_y=allsum_y/len(testset) # 전체 이미지의 y축 크기 평균
print (avg_x,avg_y,max_x,max_y) # x축 크기 평균, y축 크기 평균, 최대 x축 크기, 최대 y축 크기 출력


221.51 243.43 769 834


평균 : (x,y) =(221.51, 243.43)

이미지 크기를 256 x 256 으로 조정

In [4]:
# 문자열의 좌표값을 정수형 좌표값으로 변환하는 함수
def convert_str_to_int(coors):
    coor=np.zeros([len(coors),14,2],dtype=int) # 변환한 좌표를 저장하기 위한 배열 초기화
    for i in range(len(coors)): # 주어진 갯수 만큼 반복
        for j in range(14): # 좌표 개수 만큼 반복
            a=0 # 컴마 위치 확인용
            # "," 위치 체크
            while True: # 무한 반복문
                if coors[i][j+1][a]=="," : break # 해당 문자가 컴마면 무한 반복문 탈출
                else : a+=1 # 컴마가 아니면 다음 문자로 넘어감
            # ,를 기준으로 각각의 좌표 x y를 나누어 int로 형변환 하여 저장
            coor[i][j]=int(coors[i][j+1][1:a]),int(coors[i][j+1][a+1:-1])
            
    # 이렇게 반복하여 추출한 좌표값을 반환
    return coor


In [186]:
def convert2_str_to_int(dataset):
    MET=np.zeros([len(dataset),1],dtype=float) # 변환한 좌표를 저장하기 위한 배열 초기화
    for i in range(len(dataset)): # 주어진 갯수 만큼 반복
        a=0 # ']' 위치 확인용
        # "]" 위치 체크
        #while True: # 무한 반복문
         #   j=15
         #   if dataset[i][j][a]=="]" : break # 해당 문자가 ']'면 무한 반복문 탈출
         #   else : a+=1 # ]가 아니면 다음 문자로 넘어감
         #   # ]를 기준으로 MET값을 int로 형변환 하여 저장
        MET[i]=float(dataset[i][15])
       
    # 이렇게 반복하여 추출한 좌표값을 반환
    return MET

MET = convert2_str_to_int(testset)
print(MET)

[[ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 0.7]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]
 [ 1.8]]


In [7]:
img=skimage.io.imread(testset[0][0])
print(img.shape)
coor_set=convert_str_to_int(testset)
print(coor_set.shape)
print(coor_set[0][0])
print(coor_set[0][1])
print(coor_set[0][0][0])
print(coor_set[0][0][0]*(256/img.shape[0]))

(176, 79, 3)
(100, 14, 2)
[ 17 146]
[ 23 115]
17
24.7272727273


In [8]:
def Preprocess(dataset):
    img_set=np.zeros([len(dataset),256,256,3]) 
    # 이미지를 저장해두기 위한 텐서 배열 초기화 - 이미지 갯수 / x크기 / y크기 / RGB
    
    coor_set=convert_str_to_int(dataset) 
    # 문자열의 좌표값을 정수형 좌표값으로 변환하여 저장
    
    for i in range(len(dataset)): # 주어진 데이터 갯수만큼 반복
        img=skimage.io.imread(dataset[i][0]) # 이미지 불러오기
        
        img_set[i]=skimage.transform.resize(img,(256,256)) 
        # 이미지 크기를 재조정 하여 저장
        
        #이미지 크기 변환을 하였으므로 그에 걸맞게 좌표 위치또한 재조정
        for j in range(14): #주어진 좌표 개수 만큼 반복
            if coor_set[i][j][0]==-1: coor_set[i][j][0]=-1 
                # x 좌표 값이 -1 이면 조정을 해주지 않음
            else: coor_set[i][j][0]=coor_set[i][j][0]*(256/img.shape[1]) 
                # x축 크기의 변환된 비율 만큼 곱해 Scaling 
            if coor_set[i][j][1]==-1: coor_set[i][j][1]=-1 # y 좌표 값이 -1 이면 조정을 해주지 않음
            else: coor_set[i][j][1]=coor_set[i][j][1]*(256/img.shape[0]) # y축 크기의 변환된 비율 만큼 곱해 Scaling 
    
    # 이렇게 변환한 이미지 세트와 좌표 세트를 반환
    return img_set, coor_set

In [9]:
# 전처리 함수의 입력으로 불러온 CSV 파일을 저장한 testset 변수로 입력.
# 변환된 이미지 세트와 좌표 세트를 각각 picpic과 coco에다 저장해줌.
picpic,coco=Preprocess(testset)

/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [10]:
# 각 세트의 크기 확인
print (picpic.shape,coco.shape)

(100, 256, 256, 3) (100, 14, 2)


In [11]:
# 이미지와 좌표를 받아와서 이미에 해당 좌표를 입력
# 잘 변환이 되었는지 확인하는 용도

import cv2 # 그림에 숫자 넣기
from skimage import draw # 그림에 원 그리기

def markJoints(img, joints):  
    circSize=5 # 그려질 원의 크기
    font = cv2.FONT_HERSHEY_SIMPLEX # 글씨체 설정
    for i in range(14):
        x = int(joints[i,0]) # X 좌표
        y = int(joints[i,1]) # Y 좌표
        if x!=-1: # 그림에서 보이지 않는 좌표는 찍지 않도록
            rr, cc = skimage.draw.circle(y, x, circSize) # 원 그리기  `skimage.draw.circle(r, c, 반지름[shape])`
            skimage.draw.set_color(img, (rr, cc), (1,0,0)) # 색 설정 `set_color(image, coords, color)`
            cv2.putText(img, str(i+1), (x,y), font, 0.5, (0.5,0.5,0.5), 2, cv2.LINE_AA) # 그림 ㄱ
    return img


In [12]:
#imge = np.zeros((10,10), dtype=np.uint8)
#rr, cc = circle(4,4,5)
#imge[rr, cc]=1

In [13]:
# 전처리된 이미지 저장
%mkdir p_img p_with_coors_img # 현재 디렉토리에 처리된 이미지와 좌표를 찍은 이미지들을 저장할 디렉토리 생성
for i in range(len(picpic)):
    skimage.io.imsave("./p_img/p-img_"+str(i)+".jpg",picpic[i]) # 처리된 이미지 저장
    skimage.io.imsave("./p_with_coors_img/p-img_"+str(i)+".jpg",markJoints(picpic[i],coco[i])) # 좌표 입력 이미지 저장

mkdir: cannot create directory ‘p_img’: File exists
mkdir: cannot create directory ‘p_with_coors_img’: File exists


/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [14]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)


## Linear Regression

좌표 데이터 구조 변환 (100,14,2) -> (100,28)

[x1,y1,x2,y2 ..., x14,y14]의 구조형태

In [66]:
coco=np.reshape(coco,(100,28))
coco

array([[ 55, 212,  74, ...,  56, 113,  24],
       [ 93, 190,  51, ...,  56,  59,  23],
       [101, 217,  92, ...,  58,  88,  29],
       ..., 
       [ -1,  -1,  -1, ...,  57, 164,   0],
       [ -1,  -1,  63, ...,  62, 130,   8],
       [ -1,  -1,  -1, ...,  42, 152,   6]])

In [117]:
MET

array([[ 1.8],
       [ 1.8],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 0.7],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1. ],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1

100개의 좌표와 MET데이터 중 10개를 test데이터로 사용

In [242]:
a=0;b=0
coco_train=np.zeros([90,28])
MET_train=np.zeros([90,1])
coco_test=np.zeros([10,28])
MET_test=np.zeros([10,1])

for i in range (100):
    if i % 10 == 0:
        coco_test[a]=coco[i]
        MET_test[a]=MET[i]
        a+=1
    else :
        coco_train[b]=coco[i]
        MET_train[b]=MET[i]
        b+=1

In [420]:
MET_test

array([[ 0.7],
       [ 0.7],
       [ 1. ],
       [ 1. ],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.8],
       [ 1.8]])

# Normal Equation

In [243]:
X_b = np.c_[np.ones((100, 1)), coco]  # add x0 = 1 to each instance
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(MET)

In [244]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(coco_train, MET_train)
lin_reg.intercept_, lin_reg.coef_

(array([ 1.34233428]),
 array([[  7.73637896e-04,  -1.42850226e-03,  -2.49566863e-03,
           1.11523417e-03,   4.87206343e-04,  -7.25978573e-04,
          -3.94651425e-05,   1.37655299e-03,   1.76884740e-03,
          -1.90003530e-03,  -4.94357481e-04,   1.84004983e-04,
           7.21798714e-04,  -2.41685810e-04,   2.20163579e-03,
          -4.62483315e-04,  -1.02653960e-03,   3.86028080e-04,
          -2.91114312e-04,  -4.09832508e-03,   2.71276402e-04,
           7.26254711e-05,  -1.24677222e-04,   1.15640578e-03,
          -6.70050881e-04,  -3.01324297e-05,   2.45874033e-04,
           3.41700822e-03]]))

In [245]:
lin_reg.score(coco_train,MET_train)

0.35585022613710404

In [246]:
lin_reg.score(coco_test,MET_test)

-0.16036131087619077

In [247]:
lin_reg.predict(coco_test)

array([[ 1.33747336],
       [ 1.07030851],
       [ 0.924024  ],
       [ 0.74068148],
       [ 0.94442546],
       [ 1.62264384],
       [ 0.87945228],
       [ 1.0211907 ],
       [ 1.33747336],
       [ 1.25712942]])

# SDG regression

In [251]:
theta_path_sgd = []
m = len(X_b)
np.random.seed(42)

In [252]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=50, penalty=None, eta0=0.1, random_state=42)
#sgd_reg.fit(X, y.ravel())
sgd_reg.fit(coco_train, MET_train.ravel())

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.1,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=50, n_iter=None, penalty=None,
       power_t=0.25, random_state=42, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [253]:
sgd_reg.intercept_, sgd_reg.coef_

(array([  2.11858469e+10]),
 array([ -1.35354973e+13,  -5.17919313e+12,  -2.61854980e+12,
          4.24879976e+12,   7.46584505e+12,   9.23962073e+12,
          9.20998381e+11,   5.30520970e+12,   8.07386244e+12,
          8.48770889e+12,   5.76213026e+12,  -5.11262464e+12,
         -2.58432254e+12,   7.78011305e+12,   3.09608259e+12,
          6.96455652e+12,  -6.61552546e+12,   5.66928264e+12,
         -4.16446363e+12,  -2.90528100e+12,  -2.11760138e+12,
         -4.51745824e+12,   6.55242170e+12,   4.59499495e+12,
          1.64130590e+13,   5.67738562e+11,   1.52841242e+12,
         -1.60596133e+13]))

In [254]:
sgd_reg.predict(coco_test)

array([  5.94714895e+15,   5.98786813e+15,   3.60535831e+15,
         1.44112664e+15,   3.85080300e+15,   8.48004671e+14,
         6.89914178e+15,   5.68251769e+15,   5.94714895e+15,
         4.91387148e+15])

# Polynomial Regression

In [155]:

from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(coco_train)

In [156]:
X_poly[0]

array([ -1.00000000e+00,  -1.00000000e+00,  -1.00000000e+00,
        -1.00000000e+00,   1.44000000e+02,   1.18000000e+02,
        -1.00000000e+00,  -1.00000000e+00,  -1.00000000e+00,
        -1.00000000e+00,   2.03000000e+02,   1.20000000e+02,
         1.21000000e+02,   9.00000000e+01,   1.27000000e+02,
         6.30000000e+01,   1.39000000e+02,   2.60000000e+01,
         2.01000000e+02,   4.70000000e+01,   1.99000000e+02,
         1.06000000e+02,   1.84000000e+02,   1.45000000e+02,
         1.60000000e+02,   4.20000000e+01,   1.52000000e+02,
         6.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,  -1.44000000e+02,
        -1.18000000e+02,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,  -2.03000000e+02,
        -1.20000000e+02,  -1.21000000e+02,  -9.00000000e+01,
        -1.27000000e+02,  -6.30000000e+01,  -1.39000000e+02,
        -2.60000000e+01,  -2.01000000e+02,  -4.70000000e+01,
        -1.99000000e+02,

In [158]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, MET_train)
lin_reg.intercept_, lin_reg.coef_

(array([ 1.97887375]),
 array([[ -1.57470937e-07,  -1.68002684e-07,  -1.81452358e-07,
          -1.52731274e-07,  -9.03247355e-08,  -2.83350242e-08,
          -1.04258555e-07,  -2.90847503e-08,  -2.23510694e-07,
          -2.11549485e-07,  -2.11876345e-07,  -1.94924908e-07,
          -3.73381714e-08,  -1.47080508e-08,  -1.99722882e-08,
           8.58041924e-09,  -2.58589647e-08,  -3.29159350e-09,
          -5.73211660e-08,  -1.12389932e-08,  -5.50732992e-08,
          -6.52865873e-08,  -3.31502318e-08,  -8.45064327e-08,
          -4.21815204e-08,   9.35769447e-09,  -6.64432697e-08,
           1.70283709e-08,  -2.06934170e-05,  -5.00768375e-07,
          -1.19503597e-05,   4.43384767e-06,   3.40259948e-06,
           5.13070895e-06,   6.90008002e-06,  -1.34628330e-06,
           1.17937230e-05,   6.88947068e-06,   1.07137413e-05,
           3.88435739e-06,   9.66910495e-07,  -1.56504177e-06,
          -1.78478494e-06,   3.61851626e-06,   9.23817117e-07,
           4.49148523e-06,   4.8

In [191]:
lin_reg.predict(coco_test)

ValueError: shapes (10,28) and (434,1) not aligned: 28 (dim 1) != 434 (dim 0)

# MLPRegression

In [259]:
import numpy as np
from sklearn.neural_network import MLPRegressor 

X = coco_train
y = MET_train.astype(np.float)

per_clf = MLPRegressor(random_state=42, activation="relu", solver="adam")
per_clf.fit(X, y)

y_pred = per_clf.predict(coco_test)

/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ejjchl77/eunjivenv/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [272]:
MET_test

array([[ 0.7],
       [ 0.7],
       [ 1. ],
       [ 1. ],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.2],
       [ 1.8],
       [ 1.8]])

In [260]:
y_pred

array([  1.11050723,   8.21453358,  -3.2450138 ,   6.32597874,
        -3.79992467,  21.43147439,  12.07348664,   7.89122505,
         1.11050723,  -2.09279549])

# TensorFlow

In [373]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [374]:
import tensorflow as tf

n_inputs = 28 
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 1     #MET값 1개

In [375]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [376]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [377]:
X.get_shape()[1]

Dimension(28)

In [378]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons], dtype=tf.float32), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [380]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")


In [381]:
from sklearn.metrics import mean_squared_error

with tf.name_scope("loss"):
    #xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
    #                                                          logits=logits)
    loss = tf.losses.mean_squared_error(y, logits)
    #loss = -tf.reduce_sum(logits * tf.log(p))
    #loss = tf.reduce_mean(xentropy, name="loss")

In [382]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [392]:
#with tf.name_scope("eval"):
#    correct = tf.nn.in_top_k(logits, y, 1)  -> 위에서부터 가장 큰 값만 찾는건 라벨링 된 이미지 처리할때
#    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [391]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [397]:
n_epochs = 500

In [404]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: coco_train, y: MET_train})
        #acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        #acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
        #                                    y: mnist.validation.labels})
        print(epoch, "loss :", loss)

    save_path = saver.save(sess, "./dnn0319.ckpt")

0 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
1 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
2 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
3 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
4 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
5 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
6 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
7 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
8 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
9 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
10 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
11 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
12 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
13 loss : Tensor("mean_squared_error_1/value:0",

223 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
224 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
225 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
226 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
227 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
228 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
229 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
230 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
231 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
232 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
233 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
234 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
235 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
236 loss : Tensor("mean_s

373 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
374 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
375 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
376 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
377 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
378 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
379 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
380 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
381 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
382 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
383 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
384 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
385 loss : Tensor("mean_squared_error_1/value:0", shape=(), dtype=float32)
386 loss : Tensor("mean_s

In [418]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = coco_test
    y_pred = logits.eval(feed_dict={X: X_new_scaled})
    #y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [414]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", MET_test)

Predicted classes: [[ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]]
Actual classes:    [[ 0.7]
 [ 0.7]
 [ 1. ]
 [ 1. ]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.2]
 [ 1.8]
 [ 1.8]]


# MNIST

In [19]:
import tensorflow as tf
from numpy.random import multivariate_normal, permutation
from pandas import DataFrame, Series

np.random.seed(20180319)

소프트맥스 함수에 의한 확률 p 계산식을 준비한다.

In [29]:

x = tf.placeholder(tf.float32, [None, 28])
w = tf.Variable(tf.zeros([28, 11]))
w0 = tf.Variable(tf.zeros([11]))
f = tf.matmul(x, w) + w0
p = tf.nn.softmax(f)

오차 함수 loss와 트레이닝 알고리즘 train_step을 준비한다.

In [30]:
t = tf.placeholder(tf.float32, [None, 11])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer().minimize(loss)

정답률 accuracy를 정의한다.

In [36]:
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

세션을 준비하고 Variable을 초기화한다.

In [23]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

파라미터 최적화를 2000회 반복한다.

1회 실행할 때마다 트레이닝 세트에서 추출한 100개의 데이터를 이용해 경사 하강법을 적용한다.

최종적으로 테스트 세트에 대해 약 92%의 정답률을 얻을 수 있다.

#### coco의 구조 변경

    [100,14,2]를 [100,14]로 바꾸고 [x,y]를 14안에 포함하도록...좌표로 표현 가능?

In [24]:
for i in range()

SyntaxError: invalid syntax (<ipython-input-24-f8d1be0104ca>, line 1)

In [ ]:
i = 0
for _ in range(2000):
    i += 1
    coco_x=coco
    MET_t=MET
    sess.run(train_step, feed_dict={x: coco_x, t: MET_t})
#    if i % 100 == 0:
#        loss_val, acc_val = sess.run([loss, accuracy],
#            feed_dict={x:mnist.test.images, t: mnist.test.labels})
#        print ('Step: %d, Loss: %f, Accuracy: %f'
#               % (i, loss_val, acc_val))